In [3]:
import os
import sys
import glob
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
%matplotlib inline
%precision 4
plt.style.use('ggplot')


**参考**

[SciPy 官方教程 -- 线性代数](http://docs.scipy.org/doc/scipy/reference/tutorial/linalg.html)

# 大型线性系统

这是大数据的时代。每一天的每一秒，全世界无数系统中的数据都会被记录。我们的购物习惯、书籍和电影的喜好、键入电子邮件信息的关键字、药品记录、拨打电话的 NSA 记录、基因数据 -- 并且不经过分析它们就毫无用处。

海量的数据集在处理中会有巨大的挑战。求解 $10$ 个未知数的 $10$ 个方程组很简单，并且我们不需要关心什么方式。但是随着系统规模的增长，算法复杂度和效率就变得十分重要了。

## 示例：Netflix 竞赛（约 2006 ~ 2009）

更加详细的描述：
    
http://en.wikipedia.org/wiki/Netflix_Prize
        
完整的技术故事：

http://www.stat.osu.edu/~dmsl/GrandPrize2009_BPC_BigChaos.pdf

2006 年，Netflix 举办了一场竞赛，其中它提供了 $18,000$ 个电影的多于 $400,000$ 个评分。目标是预测用户对一个电影的评分，基于之前的评分*和*“相似”用户的评分。这个任务总需要分析 $400,000\times 18,000$ 个矩阵。上面的维基百科链接描述了这个竞赛，第二个链接是该方法的非常详细的描述（它考虑到了用户的重要特征，例如用户的口味如何随时间改变）。这个分析部分和矩阵分解相关 -- 我们不会深入获胜算法的细节，但是我们会在基本的矩阵分解上花一些时间。

# 矩阵分解

在使用计算高效方式求解线性方程组时，矩阵分解是个重要的步骤。

## LU 分解和高斯消元

LU 表示 'Lower Upper'，所以矩阵 $A$ 的 LU 分解是这样的分解：
$$A= LU$$
其中 $L$ 是下三角矩阵，$U$ 是上三角矩阵。

现在， LU 分解本质上是高斯消元，但是我们我们只处理矩阵 $A$（而不是增广矩阵）。

让我们回顾高斯消元（GE）如何工作。出于简洁性，我们会处理 $3\times 3$ 的方程组，但是这里的任何东西都可以推广到 $n\times n$ 的情况。考虑下面的方程组：

$$\left(\begin{matrix}a_{11}&a_{12} & a_{13}\\a_{21}&a_{22}&a_{23}\\a_{31}&a_{32}&a_{33}\end{matrix}\right)\left(\begin{matrix}x_1\\x_2\\x_3\end{matrix}\right) = \left(\begin{matrix}b_1\\b_2\\b_3\end{matrix}\right)$$

出于简洁性，让我们假设最左侧的矩阵 $A$ 是非奇异的。为了使用 GE 求解这个方程组，我们从“增广矩阵”开始：

$$\left(\begin{array}{ccc|c}a_{11}&a_{12} & a_{13}& b_1 \\a_{21}&a_{22}&a_{23}&b_2\\a_{31}&a_{32}&a_{33}&b_3\end{array}\right)$$

我们从第一个元素 $a_{11}$ 开始。如果 $a_{11} \neq 0$，我们将第一行除以 $a_{11}$，之后从其它每一行减去第一行的适当倍数，使所有行的第一个元素变为零。（如果 $a_{11}$ 是零，我们需要交换行。我们不会在这里深入这个细节。）结果是这样：



$$\left(\begin{array}{ccc|c}
1 & \frac{a_{12}}{a_{11}} & \frac{a_{13}}{a_{11}} & \frac{b_1}{a_{11}} \\
0 & a_{22} - a_{21}\frac{a_{12}}{a_{11}} & a_{23} - a_{21}\frac{a_{13}}{a_{11}}  & b_2 - a_{21}\frac{b_1}{a_{11}}\\
0&a_{32}-a_{31}\frac{a_{12}}{a_{11}} & a_{33} - a_{31}\frac{a_{13}}{a_{11}}  &b_3- a_{31}\frac{b_1}{a_{11}}\end{array}\right)$$

我们对第二行重复这个过程，首先除以开头的元素，之后从第三行和第一行减去结果的适当倍数，使第三行和第一行的第二个元素为零。我们继续这个过程，直到左边成为单位矩阵。这时，我们就可以“读出”解了：也就是，右侧的向量 $x$ 就是产生的列向量。通常，停在*简化的行阶梯形*（上三角，对角线上元素为一）更加有效，之后*回代*来获得最终结果。

要注意有些时候，交换行来获得简化的行阶梯性非常必要。这就叫做*部分主元消去*。如果我们也操作列，就叫*完全主元消去*。

需要提及的是，我们可以使用 GE 获得矩阵的逆，通过将单元矩阵作为增广部分，并将矩阵 $A$ 化简为单位矩阵。（译者注，也就是 $(A|I) \sim (I|A^{-1})$。）

现在，当我们一次求解一个方程组，并且结果 $b$ 只有一个时，非常有效。许多应用涉及到求解多个问题，其中我们左侧的矩阵方程不变，但是结果向量 $b$ 有很多个。这里，*分解* $A$ 会更加高效。

首先，我们以 GE 开始，但是我们保留消元所需的不同倍数的“踪迹”。例如，考虑这个矩阵

$$A = \left(\begin{matrix} 1 & 3 & 4 \\
                           2& 1& 3\\
                           4&1&2
                           \end{matrix}\right)$$

我们需要将第二行减去第一行的两倍，来消去第二行的第一个元素。之后将第三行减去第一行的四倍。我们不把第二行和第三行的第一个元素变为零，而是记录消元所需的倍数，所以：

$$\left(\begin{matrix} 1 & 3 & 4 \\
                           (2)& -5 & -5\\
                           (4)&-11&-14
                           \end{matrix}\right)$$


之后我们消去第三行的第二个元素：


$$\left(\begin{matrix} 1 & 3 & 4 \\
                           (2)& -5 & -5\\
                           (4)&(\frac{-11}{5})&-3
                           \end{matrix}\right)$$
                           
现在我们就完成了分解：
$$L= \left(\begin{matrix} 1 & 0 & 0 \\
                           2& 1 & 0\\
                           4&\frac{-11}5&1
                           \end{matrix}\right)
                          U = \left(\begin{matrix} 1 & 3 & 4 \\
                           0& -5 & -5\\
                           0&0&-3
                           \end{matrix}\right)$$
                                                  

我们可以通过求解两个回代问题来求解方程组：
    
$$Ly = b$$    and
$$Ux=y$$


这些都是 $O(n^2)$ 的，所以当有多个结果需要求解时，分解会更加高效。

让我们使用 NumPy 来实现：

In [4]:
import numpy as np
import scipy.linalg as la
np.set_printoptions(suppress=True) 

A = np.array([[1,3,4],[2,1,3],[4,1,2]])

print(A)

P, L, U = la.lu(A)
print(np.dot(P.T, A))
print
print(np.dot(L, U))
print(P)
print(L)
print(U)

[[1 3 4]
 [2 1 3]
 [4 1 2]]
[[ 4.  1.  2.]
 [ 1.  3.  4.]
 [ 2.  1.  3.]]

[[ 4.  1.  2.]
 [ 1.  3.  4.]
 [ 2.  1.  3.]]
[[ 0.  1.  0.]
 [ 0.  0.  1.]
 [ 1.  0.  0.]]
[[ 1.      0.      0.    ]
 [ 0.25    1.      0.    ]
 [ 0.5     0.1818  1.    ]]
[[ 4.      1.      2.    ]
 [ 0.      2.75    3.5   ]
 [ 0.      0.      1.3636]]


要注意 NumPy 使用*部分主元消去*来实现分解（使用最大的主元来交换矩阵的行）。这是由于较小主元可能导致数值不稳定。这也是应该尽可能使用库函数的另一个原因。

## Cholesky 分解

回忆一下，如果对于任何非零 n 维向量 $u$，

$$u^TA u > 0$$

则方阵 $A$ 是正定的，

并且，对称的正定阵 $A$ 是个正定阵，满足：

$$A = A^T$$

假设 $A$ 是个对称的正定阵。存在这样的唯一分解：

$$A = L L^T$$

其中 $L$ 是下三角矩阵，对角线元素为正，$L^T$ 是它的转置。这个分解称为 Cholesky 分解，$L$ 可以解释为 $A$ 的“平方根”。

### 算法：

假设 $A$ 是 $n\times n$ 的矩阵。我们使用下面的迭代过程来寻找矩阵 $L$：


$$A = \left(\begin{matrix}a_{11}&A_{12}\\A_{12}&A_{22}\end{matrix}\right) =
\left(\begin{matrix}\ell_{11}&0\\
L_{12}&L_{22}\end{matrix}\right)
\left(\begin{matrix}\ell_{11}&L_{12}\\0&L_{22}\end{matrix}\right)
$$

1.) 令 $\ell_{11} = \sqrt{a_{11}}$

2.) $L_{12} = \frac{1}{\ell_{11}}A_{12}$

3.) 求解 $A_{22} - L_{12}L_{12}^T = L_{22}L_{22}^T$ 中的 $L_{22}$

### Example:

$$A = \left(\begin{matrix}1&3&5\\3&13&23\\5&23&42\end{matrix}\right)$$

$$\ell_{11} = \sqrt{a_{11}} = 1$$

$$L_{12} = \frac{1}{\ell_{11}} A_{12} = A_{12}$$

$\begin{eqnarray*}
A_22 - L_{12}L_{12}^T &=& \left(\begin{matrix}13&23\\23&42\end{matrix}\right) - \left(\begin{matrix}9&15\\15&25\end{matrix}\right)\\
&=& \left(\begin{matrix}4&8\\8&17\end{matrix}\right)\\
&=& \left(\begin{matrix}2&0\\4&\ell_{33}\end{matrix}\right) \left(\begin{matrix}2&4\\0&\ell_{33}\end{matrix}\right)\\
&=& \left(\begin{matrix}4&8\\8&16+\ell_{33}^2\end{matrix}\right)
\end{eqnarray*}$

所以我们可以推出 $\ell_{33}=1$。


这就产生了分解：


$$\left(\begin{matrix}1&3&5\\3&13&23\\5&23&42\end{matrix}\right) = 
\left(\begin{matrix}1&0&0\\3&2&0\\5&4&1\end{matrix}\right)\left(\begin{matrix}1&3&5\\0&2&4\\0&0&1\end{matrix}\right)$$


现在使用 NumPy：

In [5]:
A = np.array([[1,3,5],[3,13,23],[5,23,42]])
L = la.cholesky(A)
print(np.dot(L.T, L))

print(L)
print(A)

[[  1.   3.   5.]
 [  3.  13.  23.]
 [  5.  23.  42.]]
[[ 1.  3.  5.]
 [ 0.  2.  4.]
 [ 0.  0.  1.]]
[[ 1  3  5]
 [ 3 13 23]
 [ 5 23 42]]


Cholesky 分解的速度大约是 LU 分解的两倍（虽然二者的复杂度都是 $n^3$）。

## 用于 PCA 和最小二乘法的矩阵分解

## 特征值分解

### 特征向量和特征值

首先回忆，矩阵 $A$ 的*特征向量*是一个非零向量 $v$，对于一些标量 $\lambda$，满足：

$$Av = \lambda v$$

值 $\lambda$ 就叫做 $A$ 的*特征值*。

如果 $n\times n$ 的矩阵 $A$ 拥有 $n$ 个线性无关的特征向量，那么 $A$ 就可以用这种方式分解：

$$A = B\Lambda B^{-1}$$

其中 $\Lambda$ 是一个对角矩阵，对角线上的元素是 $A$ 的特征值，$B$ 的列向量是 $A$ 的对应特征向量。

定理：

* $n\times n$ 的矩阵是可对角化的 $\iff$ 它有 $n$ 个线性无关的特征向量。
* 对称的正定阵只拥有正的特征值，并且它的特征值分解
$$A=B\Lambda B^{-1}$$

通过正交变换 $B$ 来实现。（也就是，它的特征向量是标准正交集）



#### 计算特征值

从定义中很容易看出，如果 $v$ 是 $n\times n$ 矩阵 $A$ 的特征向量，拥有特征值 $\lambda$，那么

$$(A - \lambda I)v = \bf{0}$$

其中 $I$ 是 $n$ 维单位矩阵，$\bf{0}$ 是 n 维零向量。因此，$A$ 的特征值满足：

$$\det\left(A-\lambda I\right)=0$$

上式的左边是 $\lambda$ 的多项式，叫做 $A$ 的*特征多项式*。所以，为了找到 $A$ 的特征值，我们需要找到特征多项式的根。



但是，从计算上来说，计算特征多项式以及求解根，是极其复杂的。因此，在实践中，会使用数值方法 -- 来找到特征值以及相应的特征向量。我们不会陷入用于计算特征值的算法细节，这里是个 NumPy 的示例：

In [6]:
A = np.array([[0,1,1],[2,1,0],[3,4,5]])

u, V = la.eig(A)
print(np.dot(V,np.dot(np.diag(u), la.inv(V))))
print(u)


[[-0.+0.j  1.+0.j  1.+0.j]
 [ 2.+0.j  1.+0.j  0.+0.j]
 [ 3.+0.j  4.+0.j  5.+0.j]]
[ 5.8541+0.j -0.8541+0.j  1.0000+0.j]


**注意：** 许多矩阵*无法*对角化，许多矩阵拥有*复数*特征值（即使所有元素都是实数）。

In [15]:
A = np.array([[0,1],[-1,0]])
print(A)

u, V = la.eig(A)
print(np.dot(V,np.dot(np.diag(u), la.inv(V))))
print(u)

[[ 0  1]
 [-1  0]]
[[ 0.+0.j  1.+0.j]
 [-1.+0.j  0.+0.j]]
[ 0.+1.j  0.-1.j]
[ 0.+1.j  0.-1.j]


In [8]:
# If you know the eigenvalues must be reeal 
# because A is a positive definite (e.g. covariance) matrix 
# use real_if_close

A = np.array([[0,1,1],[2,1,0],[3,4,5]])
u, V = la.eig(A)
print(u)
print np.real_if_close(u)

[ 5.8541+0.j -0.8541+0.j  1.0000+0.j]
[ 5.8541 -0.8541  1.    ]


#### 奇异值

对于任何 $m\times n$ 的矩阵 $A$，我们将它的*奇异值*定义为 $A^TA$ 特征值的平方根。这是一个良好的定义，因为 $A^TA$ 永远是对称，并且正定的，所以它的特征值都是正实数。奇异值是矩阵的非常重要的属性。几何上，矩阵 $A$ 将 $\mathbb{R}^n$ 中的单位球映射为椭圆。 奇异值是半轴的长度。

奇异值也提供了矩阵*稳定性*的度量。我们会在这篇文章末尾再次看到它。

## QR 分解

就像上一个分解那样，$QR$ 分解是一种方式，将矩阵 $A$ 写成两个形式更简单的矩阵乘积。这里，我们希望：

$$ A= QR$$
其中 $Q$ 是 $m\times n$ 的矩阵，满足 $Q Q^T = I$（即 $Q$ 是正交的），并且 $R$ 是 $n\times n$ 的上三角矩阵。

这实际上是 $A$ 的列向量的 Gram-Schmidt 正交化的矩阵形式。G-S 算法本身是不稳定的，所以有多种其它算法被开发出来，来计算 QR 分解。我们不会详细涉及它们，因为它们有些超出我们的范围。

$Q$ 的前 $k$ 列 是 $A$ 的前 $k$ 列列空间的标准正交基。

可迭代的 QR 分解通常用于计算特征值。 

## 奇异值分解

另一种重要的矩阵分解是奇异值分解（SVD）。对于任何 $m\times n$ 的矩阵 $A$，我们可以写出：

$$A= UDV$$

其中 $U$ 是 $m\times m$ 的酉矩阵（实数情况下正交），$D$ 是 $m\times n$ 的对角长方阵，其对角线上的元素 $d_1,...,d_m$ 都是非负的。 $V$ 是 $n\times n$ 的酉矩阵（正交）。SVD 用于主成分分析，以及 Moore-Penrose 广义逆矩阵的计算。

## 稳定性和条件数

数值算法是*稳定的*和*高效的*十分重要。效率是算法的一个属性，但是稳定性是系统自身的一个属性。

### 示例

$$\left(\begin{matrix}8&6&4&1\\1&4&5&1\\8&4&1&1\\1&4&3&6\end{matrix}\right)x = \left(\begin{matrix}19\\11\\14\\14\end{matrix}\right)$$

In [9]:
A = np.array([[8,6,4,1],[1,4,5,1],[8,4,1,1],[1,4,3,6]])
b = np.array([19,11,14,14])
la.solve(A,b)

array([ 1.,  1.,  1.,  1.])

In [10]:
b = np.array([19.01,11.05,14.07,14.05])
la.solve(A,b)

array([-2.34 ,  9.745, -4.85 , -1.34 ])

要注意向量 $b$ 中的*微小*波动，会造成解中的非常大的差异！这种情况发生时，我们就说矩阵 $A$ 是病态的。当矩阵非常接近于奇异矩阵（也就是不可逆）时，会发生这种情况。

### 条件数

  

这种行为的一个度量叫做*条件数*。它定义为：

$$ cond(A) = ||A||\cdot ||A^{-1}|| $$

通常，它比较难以计算。

定理：

$$cond(A) = \frac{\lambda_1}{\lambda_n}$$

其中 $\lambda_1$ 是 $A$ 的最大奇异值，$\lambda_n$ 是最小奇异值。条件数越大，系统越不稳定。通常，如果最小值和最大值之间的差距很大，条件数就很大。

#### 示例

In [11]:
U, s, V = np.linalg.svd(A)
print(s)
print(max(s)/min(s))

[ 15.5457   6.9002   3.8363   0.0049]
3198.6725812


#### 预处理

我们有时可以通过“预处理”来改善这个行为。我们不求解
$$Ax=b$$
而是求解
$$D^{-1}Ax=D^{-1}b$$
其中 $D^{-1}A$ 的条件数比 $A$ 本身要小。

预处理是个*非常*难懂的话题，超出了这个课程的范围。这里提及它，只是为了让你意识到这样的东西存在，你应该曾经碰到过病态问题。 

<font color=red>练习</font>
----

**1**. 手动并使用 NumPy 计算下列矩阵的 LU 分解

$$\left(\begin{matrix}1&2&3\\2&-4&6\\3&-9&-3\end{matrix}\right)$$


In [12]:
# Your code here

**2**. 手动并使用 NumPy 计算下列矩阵的 Cholesky 

$$\left(\begin{matrix}1&2&3\\2&-4&6\\3&6&-3\end{matrix}\right)$$


In [13]:
# Your code here

**3**. 使用 Python 编写函数，使用 SVD 分解来求解方程组

$$Ax = b$$

你的函数应该接受 $A$ and $b$ 作为输入，并返回 $x$。

你的函数应包含下列操作：

* 首先，检查 $A$ 是否可逆 - 不是的话返回错误信息
* 使用 SVD 对 $A$ 求逆，并求解方程组
* 返回 $x$

测试你的函数的正确性。

In [14]:
# Your code here